In [1]:
import boto3, os

In [2]:
ec2 = boto3.resource('ec2')

In [3]:
# We will use the general SSH security group.
# Determine if it already exists.
security_group_name = 'SSH-security-group'
security_group_iterator = ec2.security_groups.filter(
    GroupNames=[
        security_group_name
    ]
)
try:
    # If this causes an exception, create the security group
    list(security_group_iterator)
except:
    security_group = ec2.create_security_group(
        GroupName=security_group_name,
        Description='Security group to SSH into instances'
    )
    response = security_group.authorize_ingress(
        IpPermissions=[
            {'FromPort': 22,
            'ToPort': 22,
            'IpProtocol': 'tcp',
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
            }
        ]
    )

In [4]:
# We need a key pair to SSH into the instance
# Determine if it already exists
key_pair_name = 'basic-launch-key-pair'
key_pair_info_iterator = ec2.key_pairs.filter(
    KeyNames=[
        key_pair_name
    ]
)
try:
    # If this causes an exception, create the key pair
    list(key_pair_info_iterator)
except:
    key_pair = ec2.create_key_pair(
        KeyName=key_pair_name
    )
    key_pair_file_name = '/Users/dougjam/.ssh/' + key_pair_name + '.pem'
    with open(key_pair_file_name, 'w') as f:
        f.write(key_pair.key_material)
        f.close()
    os.chmod(key_pair_file_name, 0o600)

In [5]:
# create the basic instance
instances = ec2.create_instances(
    ImageId='ami-00dfe2c7ce89a450b',
    InstanceType='t2.micro',
    KeyName=key_pair_name,
    MaxCount=1,
    MinCount=1,
    SecurityGroups=[
        security_group_name
    ]
)
instance = instances[0]
instance.wait_until_running()
instance.reload()

In [6]:
# add the public ip address to route53 so SSH is easier
route53 = boto3.client('route53')

# it seems like waiting will be the right thing to do
route53_waiter = route53.get_waiter('resource_record_sets_changed')
response = route53.list_hosted_zones_by_name(
    DNSName='doug-nicholson.net'
)
zone_id = response['HostedZones'][0]['Id'][12:]
response = route53.change_resource_record_sets(
    HostedZoneId=zone_id,
    ChangeBatch={
        'Changes': [
            {
                'Action': 'CREATE',
                'ResourceRecordSet': {
                    'Name': 'basic.doug-nicholson.net',
                    'Type': 'A',
                    'TTL': 300,
                    'ResourceRecords': [
                        {
                            'Value': instance.public_ip_address
                        },
                    ],
                }
            }
        ]
    }
)
route53_waiter.wait(
    Id=response['ChangeInfo']['Id'][8:]
)